<a href="https://colab.research.google.com/github/Simongazze/Tesina_de_grado/blob/main/Web_scraping_play_by_play.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!git clone https://github.com/Simongazze/Tesina_de_grado.git
%cd Tesina_de_grado
!ls

Cloning into 'Tesina_de_grado'...
remote: Enumerating objects: 417, done.
remote: Counting objects: 100% (417/417), done.
remote: Compressing objects: 100% (33/33), done.
remote: Total 417 (delta 376), reused 411 (delta 376), pack-reused 0 (from 0)
Receiving objects: 100% (417/417), 3.97 MiB | 21.48 MiB/s, done.
Resolving deltas: 100% (376/376), done.
/content/Tesina_de_grado
 Anteproyecto-de-tesina---Simón-Gazze.pdf    Tesina_de_grado.Rproj
'Anteproyecto de tesina - Simón Gazze.Rmd'   Web-scraping
 README.md


In [2]:
# Instalar Google Chrome y ChromeDriver
!apt-get update -y
!wget -q https://dl.google.com/linux/direct/google-chrome-stable_current_amd64.deb
!dpkg -i google-chrome-stable_current_amd64.deb || apt-get -fy install -y
!pip install selenium webdriver-manager


Get:1 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Hit:2 https://cli.github.com/packages stable InRelease
Hit:3 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:4 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease [1,581 B]
Get:5 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Get:6 https://r2u.stat.illinois.edu/ubuntu jammy InRelease [6,555 B]
Get:7 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,632 B]
Get:8 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease [18.1 kB]
Get:9 http://archive.ubuntu.com/ubuntu jammy-backports InRelease [127 kB]
Hit:10 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Get:11 http://security.ubuntu.com/ubuntu jammy-security/main amd64 Packages [3,528 kB]
Hit:12 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Get:13 http://security.ubuntu.com/ubuntu jammy-security/restricted amd64 Packag

In [3]:
import pandas as pd
import re # Carta de trio
import json # Box score
import time # Box score

from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from webdriver_manager.chrome import ChromeDriverManager
from bs4 import BeautifulSoup

from tqdm import tqdm

pd.set_option('display.max_columns', None)

Link de los partidos

In [4]:

# Nombre del archivo de texto que contiene el HTML
file_name = "Web-scraping/lnb-web-scraping/data/external/elemento_html_partidos_temporada_24_25.txt"

# Diccionario para almacenar los datos de los partidos
match_data = {}
try:
    # 1. Leer el contenido del archivo .txt
    with open(file_name, "r", encoding="utf-8") as f:
        html_content = f.read()

    # 2. Crear un objeto BeautifulSoup para parsear el HTML
    soup = BeautifulSoup(html_content, "html.parser")

    # 3. Buscar todas las filas de la tabla (<tr>)
    filas_partidos = soup.find_all("tr", role="row")

    print(f"Extrayendo datos de {len(filas_partidos)} partidos...")

    for fila in filas_partidos:
        # Extraer los datos de las celdas (<td>) de cada fila
        celdas = fila.find_all("td")

        # Asegurarse de que la fila tiene la estructura esperada
        if len(celdas) > 8:
            # Extraer los datos por su índice de celda
            fecha_hora = celdas[0].get_text(strip=True)[-17:]
            nombre_local = celdas[1].get_text(strip=True)
            puntos_local = celdas[3].get_text(strip=True)
            puntos_visita = celdas[4].get_text(strip=True)
            nombre_visita = celdas[6].get_text(strip=True)

            # El link está dentro de la celda en el índice 8 (anteriormente 9)
            link_tag = celdas[8].find("a", href=True)
            link_estadisticas = link_tag.get('href') if link_tag else None

            # Usar una combinación única como clave del diccionario
            match_key = f"{nombre_local} vs {nombre_visita} ({fecha_hora})"

            # Guardar los datos en el diccionario
            match_data[match_key] = {
                "nombre_local": nombre_local,
                "puntos_local": puntos_local,
                "nombre_visita": nombre_visita,
                "puntos_visita": puntos_visita,
                "link_estadisticas": link_estadisticas
            }

    print("\nDiccionario de datos de partidos creado.")

    # Mostrar el diccionario para su verificación
    display(match_data)

except FileNotFoundError:
    print(f"Error: No se encontró el archivo '{file_name}'. Asegúrate de que el archivo existe en la ruta correcta.")
except Exception as e:
    print(f"Ocurrió un error al procesar el archivo: {e}")

Extrayendo datos de 380 partidos...

Diccionario de datos de partidos creado.


{'ATENAS (C) vs BOCA (007/10/2024 22:10)': {'nombre_local': 'ATENAS (C)',
  'puntos_local': '69',
  'nombre_visita': 'BOCA',
  'puntos_visita': '81',
  'link_estadisticas': 'https://estadisticascabb.gesdeportiva.es/partido/rM2-eTJQHJsR2FLJYE8GRw==?a=1'},
 'OBRAS vs PLATENSE (008/10/2024 20:00)': {'nombre_local': 'OBRAS',
  'puntos_local': '94',
  'nombre_visita': 'PLATENSE',
  'puntos_visita': '90',
  'link_estadisticas': 'https://estadisticascabb.gesdeportiva.es/partido/UYiBwuNnKVL9qCbXhOY76g==?a=1'},
 'INSTITUTO vs OLIMPICO (LB) (008/10/2024 21:00)': {'nombre_local': 'INSTITUTO',
  'puntos_local': '99',
  'nombre_visita': 'OLIMPICO (LB)',
  'puntos_visita': '73',
  'link_estadisticas': 'https://estadisticascabb.gesdeportiva.es/partido/Dwgh5U3-8hNWcMtXCbn6yw==?a=1'},
 'UNION (SF) vs SAN LORENZO (008/10/2024 21:30)': {'nombre_local': 'UNION (SF)',
  'puntos_local': '68',
  'nombre_visita': 'SAN LORENZO',
  'puntos_visita': '63',
  'link_estadisticas': 'https://estadisticascabb.gesdepor

Box score

In [5]:
# --- Datos de ejemplo ---
# match_data = {
#     "ATENAS (C) vs BOCA (07/10/2024 22:10)": {
#         "link_estadisticas": "https://estadisticascabb.gesdeportiva.es/partido/rM2-eTJQHJsR2FLJYE8GRw==?a=1"
#     }
# }
# --- Fin del ejemplo ---

all_player_stats = []
driver = None

try:
    #primer_partido_key = list(match_data.keys())[0]  # Obtener el último partido del diccionario
    #primer_link = match_data[primer_partido_key]["link_estadisticas"]

    partido_key = 'FERRO vs PEÑAROL (MDP) (009/10/2024 20:00)'
    partido_info = match_data[partido_key]["link_estadisticas"]
    print(f"El enlace del partido es: {partido_info}")

    # --- Parte 1: NAVEGACIÓN CON SELENIUM (YA FUNCIONA CORRECTAMENTE) ---
    print("\nIniciando Selenium...")
    options = Options()
    options.add_argument("--headless")
    options.add_argument("--no-sandbox")
    options.add_argument("--disable-dev-shm-usage")
    options.add_argument("user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36")

    driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)
    driver.get(partido_info)

    wait = WebDriverWait(driver, 20)

    print("Cambiando al iframe principal...")
    wait.until(EC.frame_to_be_available_and_switch_to_it((By.TAG_NAME, "iframe")))
    print("Cambio al iframe principal exitoso.")

    print("Buscando y haciendo clic en la pestaña 'Estadísticas'...")
    tab_selector = (By.CSS_SELECTOR, "li.pestana-estadisticas")
    estadisticas_tab_element = wait.until(EC.visibility_of_element_located(tab_selector))
    driver.execute_script("arguments[0].click();", estadisticas_tab_element)
    time.sleep(1)

    print("Cambiando al iframe de datos de estadísticas...")
    iframe_de_datos_selector = (By.CSS_SELECTOR, "div.contenido-estadisticas.activo iframe")
    wait.until(EC.frame_to_be_available_and_switch_to_it(iframe_de_datos_selector))
    print("Acceso al iframe de datos exitoso.")

    print("Extrayendo el código HTML final...")
    html_box_scores = driver.page_source

    # --- Parte 2: EXTRACCIÓN CON BEAUTIFULSOUP (SECCIÓN CORREGIDA) ---
    print("Procesando el HTML para extraer datos de jugadores...")
    soup = BeautifulSoup(html_box_scores, "html.parser")

    # --- INICIO DE LA CORRECCIÓN ---
    # Hacemos la búsqueda de los nombres de equipo de forma segura

    equipo_local_nombre = "Local" # Valor por defecto
    equipo_visitante_nombre = "Visitante" # Valor por defecto

    # Buscamos los divs que contienen los nombres de los equipos.
    # El selector 'div.nombre-equipo' es un candidato común.
    divs_nombres = soup.find_all("div", class_="nombre-equipo")
    if len(divs_nombres) >= 2:
        equipo_local_nombre = divs_nombres[0].get_text(strip=True)
        equipo_visitante_nombre = divs_nombres[1].get_text(strip=True)
    else:
        print("Advertencia: No se encontraron los nombres de equipo con el selector 'div.nombre-equipo'. Se usarán nombres genéricos.")
    # --- FIN DE LA CORRECCIÓN ---

    tablas = soup.find_all("table", class_="tabla-estadisticas")

    if len(tablas) >= 2:
        print(f"\nExtrayendo datos de: {equipo_local_nombre}")
        for fila in tablas[0].find("tbody").find_all("tr", onclick=True):
            onclick_attr = fila["onclick"]
            match = re.search(r"(\{.*\})", onclick_attr)
            if match:
                json_str = match.group(1).replace("'", '"')
                player_data = json.loads(json_str)
                player_data['equipo'] = equipo_local_nombre
                all_player_stats.append(player_data)

        print(f"Extrayendo datos de: {equipo_visitante_nombre}")
        for fila in tablas[1].find("tbody").find_all("tr", onclick=True):
            onclick_attr = fila["onclick"]
            match = re.search(r"(\{.*\})", onclick_attr)
            if match:
                json_str = match.group(1).replace("'", '"')
                player_data = json.loads(json_str)
                player_data['equipo'] = equipo_visitante_nombre
                all_player_stats.append(player_data)

    # --- Parte 3: PRESENTACIÓN DE DATOS CON PANDAS ---
    if all_player_stats:
        df_box_scores = pd.DataFrame(all_player_stats)
        print("\n✅ DataFrame final con todos los datos extraídos:")
        df_box_scores
    else:
        print("\nNo se pudieron extraer datos de jugadores del HTML.")

except Exception as e:
    print(f"Ocurrió un error general: {e}")
finally:
    if driver:
        driver.quit()
        print("\nNavegador cerrado.")

df_box_scores = df_box_scores[['IdJugador', 'IdClub', 'IdEquipo', 'Nombre', 'NombreCompleto',
                              'Puntos', 'TirosDos', 'TirosTres', 'TirosLibres',
                              'ReboteDefensivo', 'ReboteOfensivo', 'RebotesTotales',
                              'Asistencias', 'Recuperaciones', 'Perdidas',
                              'TaponCometido','TaponRecibido', 'FaltaCometida','FaltaRecibida','Valoracion',
                              'TiempoJuego', 'CincoInicial', 'equipo']]

# Lista de las columnas que contienen diccionarios para procesar
columnas_con_diccionarios = ['TirosDos', 'TirosTres', 'TirosLibres']

# Iteramos sobre cada columna que necesitamos transformar
for columna in columnas_con_diccionarios:
    if columna in df_box_scores.columns:
        # Creamos la nueva columna para 'Aciertos'
        # La función lambda toma cada diccionario (x) de la columna y extrae el valor de 'Aciertos'
        # .get('Aciertos', 0) es una forma segura que devuelve 0 si la clave 'Aciertos' no existe
        df_box_scores[f'{columna}Aciertos'] = df_box_scores[columna].apply(lambda x: x.get('Aciertos', 0))

        # Creamos la nueva columna para 'Fallos'
        df_box_scores[f'{columna}Fallos'] = df_box_scores[columna].apply(lambda x: x.get('Fallados', 0))

# Eliminamos las columnas originales que contenían los diccionarios para limpiar el DataFrame
df_box_scores = df_box_scores.drop(columns=columnas_con_diccionarios)
df_box_scores


El enlace del partido es: https://estadisticascabb.gesdeportiva.es/partido/7FrcQXet0O0dRypYCF4kWQ==?a=1

Iniciando Selenium...
Cambiando al iframe principal...
Cambio al iframe principal exitoso.
Buscando y haciendo clic en la pestaña 'Estadísticas'...
Cambiando al iframe de datos de estadísticas...
Acceso al iframe de datos exitoso.
Extrayendo el código HTML final...
Procesando el HTML para extraer datos de jugadores...

Extrayendo datos de: FERRO
Extrayendo datos de: PEÑAROL (MDP)

✅ DataFrame final con todos los datos extraídos:

Navegador cerrado.


,IdJugador,IdClub,IdEquipo,Nombre,NombreCompleto,Puntos,ReboteDefensivo,ReboteOfensivo,RebotesTotales,Asistencias,Recuperaciones,Perdidas,TaponCometido,TaponRecibido,FaltaCometida,FaltaRecibida,Valoracion,TiempoJuego,CincoInicial,equipo,TirosDosAciertos,TirosDosFallos,TirosTresAciertos,TirosTresFallos,TirosLibresAciertos,TirosLibresFallos
0,68608,46,69144,"GONZALEZ, M.","GONZALEZ CELANO, MARTIN",0,0,0,0,0,0,0,0,0,0,0,0,00:00,False,FERRO,0,0,0,0,0,0
1,271508,46,69144,"LEZCANO, E.","LEZCANO, EMILIANO",14,4,0,4,2,0,3,1,1,4,2,6,32:07,True,FERRO,7,3,0,5,0,1
2,326914,46,69144,"LADO, M.","LADO, MOGGA CHARLES ARNADO",11,2,0,2,1,0,2,0,2,1,4,8,21:24,True,FERRO,4,4,0,0,3,1
3,324409,46,69144,"MARTINEZ, J.","MARTINEZ, JANO DAVID",5,2,1,3,1,2,1,0,0,3,0,-1,22:18,False,FERRO,1,2,1,6,0,0
4,271509,46,69144,"ROVERES, A.","ROVERES, AUGUSTO DANIEL",2,2,1,3,0,0,0,0,0,0,0,3,07:03,False,FERRO,1,0,0,2,0,0
5,74902,46,69144,"RODRIGUEZ, F.","RODRIGUEZ, FELIPE SALVADOR",3,6,0,6,0,1,0,0,0,3,0,3,20:11,True,FERRO,0,1,1,3,0,0
6,182885,46,69144,"WOLINSKY, F.","WOLINSKY, FACUNDO ARIEL",0,0,0,0,0,0,0,0,0,0,0,0,00:00,False,FERRO,0,0,0,0,0,0
7,278259,46,69144,"GALLEGOS, R.","GALLEGOS, RODRIGO AGUSTIN",19,8,1,9,5,1,2,0,0,5,4,25,32:29,True,FERRO,6,3,0,3,7,0
8,68602,46,69144,"RODRIGUEZ, C.","RODRIGUEZ, CAMILO",0,0,0,0,0,2,0,0,0,0,0,1,05:57,False,FERRO,0,1,0,0,0,0
9,325771,46,69144,"DIEZ, A.","DIEZ, ALEJANDRO GABRIEL",10,3,2,5,1,0,2,0,1,3,3,10,26:35,False,FERRO,4,3,0,0,2,0


Play by Play


Obtener un partido

In [6]:
acciones_del_partido = []
driver = None

try:
    # Asumiendo que 'match_data' ya fue creada
    # partido_key = list(match_data.keys())[0]
    # partido_link = match_data[partido_key]["link_estadisticas"]

    partido_key = 'FERRO vs PEÑAROL (MDP) (009/10/2024 20:00)'
    partido_link = match_data[partido_key]["link_estadisticas"]
    print(f"El enlace del partido es: {partido_link}")

    # --- Parte 1: NAVEGACIÓN CON SELENIUM ---
    print("\nIniciando Selenium...")
    options = Options()
    options.add_argument("--headless")  # Modo sin interfaz
    options.add_argument("--no-sandbox")
    options.add_argument("--disable-dev-shm-usage")
    options.binary_location = "/usr/bin/google-chrome"  # 🔹 le decimos explícitamente dónde está Chrome
    options.add_argument(
        "user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36"
    )

    driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)
    driver.get(partido_link)

    wait = WebDriverWait(driver, 20)

    print("Cambiando al iframe principal...")
    wait.until(EC.frame_to_be_available_and_switch_to_it((By.TAG_NAME, "iframe")))
    print("Cambio al iframe principal exitoso.")

    print("Haciendo clic en la pestaña 'En vivo'...")
    en_vivo_tab = wait.until(EC.element_to_be_clickable((By.CSS_SELECTOR, "li.pestana-en-vivo")))
    driver.execute_script("arguments[0].click();", en_vivo_tab)

    print("Cambiando al iframe anidado del Play-by-Play...")
    iframe_pbp_selector = (By.CSS_SELECTOR, "div.contenido-en-vivo div:nth-child(2) iframe")
    wait.until(EC.frame_to_be_available_and_switch_to_it(iframe_pbp_selector))
    print("Acceso al iframe final del Play-by-Play exitoso.")

    html_pbp = driver.page_source

    # --- Parte 2: EXTRACCIÓN CON BEAUTIFULSOUP (CORREGIDA) ---
    print("Procesando el HTML para extraer las acciones del partido...")
    soup = BeautifulSoup(html_pbp, "html.parser")

    contenedor_acciones = soup.find("ul", class_="listadoAccionesPartido")

    if contenedor_acciones:
        acciones = contenedor_acciones.find_all("li", class_="accion")
        print(f"Se encontraron {len(acciones)} acciones en el partido.")

        for accion in acciones:
            # Inicializamos todas las variables
            tipo_accion = jugador = cuarto = tiempo = ""
            puntos_local = None
            puntos_visita = None

            titulo_tag = accion.find("strong", class_="titulo")
            if titulo_tag:
                tipo_accion = titulo_tag.get_text(strip=True)

            spans_info = accion.find_all("span", class_="informacion")
            if len(spans_info) >= 2:
                jugador = spans_info[0].get_text(strip=True)
                tiempo_text = spans_info[1].get_text(strip=True)
                match = re.search(r"Cuarto\s*(\d+)\s*-\s*(\d{2}:\d{2}:\d{2})", tiempo_text)
                if match:
                    cuarto = match.group(1)
                    tiempo = match.group(2)

            # --- NUEVO: Extraer el marcador de la acción ---
            marcador_tag = accion.find("strong", class_="informacionAdicional")
            if marcador_tag:
                marcador_texto = marcador_tag.get_text(strip=True)
                partes_marcador = marcador_texto.split('-')
                if len(partes_marcador) == 2:
                    try:
                        puntos_local = int(partes_marcador[0].strip())
                        puntos_visita = int(partes_marcador[1].strip())
                    except ValueError:
                        pass # Si hay un error, se quedan como None

            acciones_del_partido.append({
                "cuarto": cuarto,
                "tiempo": tiempo,
                "accion": tipo_accion,
                "jugador": jugador,
                "puntos_local": puntos_local,
                "puntos_visita": puntos_visita
            })

    # --- Parte 3: Crear y mostrar el DataFrame ---
    if acciones_del_partido:
        df_acciones = pd.DataFrame(acciones_del_partido)
        print("\n✅ DataFrame con marcador en cada acción:")
        display(df_acciones.head(10))
    else:
        print("No se pudieron extraer datos del Play-by-Play.")

except Exception as e:
    print(f"Ocurrió un error general: {e}")
finally:
    if driver:
        driver.quit()
        print("\nNavegador cerrado.")

El enlace del partido es: https://estadisticascabb.gesdeportiva.es/partido/7FrcQXet0O0dRypYCF4kWQ==?a=1

Iniciando Selenium...
Cambiando al iframe principal...
Cambio al iframe principal exitoso.
Haciendo clic en la pestaña 'En vivo'...
Cambiando al iframe anidado del Play-by-Play...
Acceso al iframe final del Play-by-Play exitoso.
Procesando el HTML para extraer las acciones del partido...
Se encontraron 606 acciones en el partido.

✅ DataFrame con marcador en cada acción:


,cuarto,tiempo,accion,jugador,puntos_local,puntos_visita
0,,,FINAL DEL PERIODO5,,NaN,NaN
1,5,00:00:01,REBOTE DEFENSIVO#9,"RODRIGUEZ, FELIPE SALVADOR",NaN,NaN
2,5,00:00:02,TIRO DE 2 FALLADO,"WALLACE, DEVANTE RASHAD-KEITH",NaN,NaN
3,5,00:00:02,REBOTE OFENSIVO#24,"WALLACE, DEVANTE RASHAD-KEITH",NaN,NaN
4,5,00:00:02,TIRO LIBRE FALLADO,"THOMAS JR, MARCUS WILEY",NaN,NaN
5,5,00:00:02,TIRO LIBRE ANOTADO,"THOMAS JR, MARCUS WILEY",84.0,82.0
6,5,00:00:02,2 TIROS LIBRES PARA EL#50,"THOMAS JR, MARCUS WILEY",NaN,NaN
7,5,00:00:02,ENTRA A PISTA EL#93,"FIERRO, MARIANO",NaN,NaN
8,5,00:00:02,ABANDONA LA PISTA EL#12,"GALLEGOS, RODRIGO AGUSTIN",NaN,NaN
9,5,00:00:02,FALTA RECIBIDA,"THOMAS JR, MARCUS WILEY",NaN,NaN



Navegador cerrado.


Agregar quintetos y equipos

In [20]:
print("Iniciando el proceso completo para generar quintetos por equipo...")

try:
    # === PASO 0: Identificar equipos local y visitante ===
    print("Paso 0: Identificando equipos local y visitante...")
    partido_key = 'FERRO vs PEÑAROL (MDP) (009/10/2024 20:00)'
    nombre_local, nombre_visitante = partido_key.split(" vs ")
    nombre_visitante = nombre_visitante.split(" (")[0]
    print(f"Equipo local: {nombre_local} | Equipo visitante: {nombre_visitante}")

    # === PASO 1: Crear el mapa de jugadores desde el Box Score ===
    print("Paso 1: Creando mapa de jugadores desde 'df_box_scores'...")
    if 'df_box_scores' not in locals() or df_box_scores.empty:
        raise NameError("El DataFrame 'df_box_scores' no existe o está vacío.")

    player_to_team_map = pd.Series(df_box_scores.equipo.values, index=df_box_scores.NombreCompleto).to_dict()

    # Normalizar nombres de equipos (quita lo que está entre paréntesis)
    df_box_scores["equipo_normalizado"] = df_box_scores["equipo"].str.replace(r"\s*\(.*\)?", "", regex=True).str.strip()

    # Crear el mapa con los nombres limpios
    player_to_team_map = pd.Series(df_box_scores.equipo_normalizado.values, index=df_box_scores.NombreCompleto).to_dict()

    # === PASO 2: Generar la columna temporal 'quinteto_en_cancha' ===
    print("Paso 2: Generando lista de 10 jugadores en cancha por acción...")
    df_cronologico = df_acciones.iloc[::-1].reset_index(drop=True)

    jugadores_en_cancha = set()
    lista_de_quintetos_mixtos = []
    cuarto_actual = None  # opcional, si querés seguir rastreando el número de cuarto

    for index, row in df_cronologico.iterrows():
        accion = row.get('accion', '')
        jugador = row.get('jugador', '')
        accion_upper = accion.upper() if isinstance(accion, str) else ''
        cuarto = row.get('cuarto', None)

        # 🔁 Reiniciar SOLO si la acción indica "FINAL DEL PERIODO"
        if "FINAL DEL PERIODO" in accion_upper:
            jugadores_en_cancha = set()
            cuarto_actual = cuarto  # opcional
            # print(f"🔁 Reinicio por FINAL DEL PERIODO en fila {index}")

        # 🔹 Actualizar quién entra y sale
        elif "ENTRA A PISTA" in accion_upper or "CAMBIO-ENTRA" in accion_upper:
            if pd.notna(jugador) and jugador != '':
                jugadores_en_cancha.add(jugador)
        elif "ABANDONA LA PISTA" in accion_upper or "CAMBIO-SALE" in accion_upper:
            if pd.notna(jugador) and jugador != '':
                jugadores_en_cancha.discard(jugador)

        lista_de_quintetos_mixtos.append(sorted(list(jugadores_en_cancha)))

    df_cronologico['quinteto_en_cancha'] = lista_de_quintetos_mixtos


    # === PASO 3: Separar el quinteto mixto en local y visitante ===
    print("Paso 3: Separando la lista de 10 jugadores en quintetos por equipo...")

    def dividir_quinteto(quinteto_mixto, roster_map, equipo_local_nombre):
        quinteto_local, quinteto_visita = [], []
        for jugador in quinteto_mixto:
            equipo = roster_map.get(jugador)
            if equipo == equipo_local_nombre:
                quinteto_local.append(jugador)
            else:
                quinteto_visita.append(jugador)
        return sorted(quinteto_local), sorted(quinteto_visita)

    nuevas_columnas = df_cronologico['quinteto_en_cancha'].apply(
        lambda q: pd.Series(dividir_quinteto(q, player_to_team_map, nombre_local))
    )
    nuevas_columnas.columns = ['quinteto_local', 'quinteto_visita']

    df_cronologico = pd.concat([df_cronologico, nuevas_columnas], axis=1)

    # === PASO 4: Agregar columnas de equipo local y visitante ===
    df_cronologico["equipo_local"] = nombre_local
    df_cronologico["equipo_visitante"] = nombre_visitante

    # === PASO 4B: Identificar a qué equipo corresponde cada acción ===
    print("Paso 4B: Identificando equipo responsable de cada acción...")

    def identificar_equipo(row):
        jugador = row.get("jugador")
        if pd.notna(jugador) and jugador in player_to_team_map:
            return player_to_team_map[jugador]
        accion = row.get("accion", "").upper()
        if "LOCAL" in accion:
            return nombre_local
        elif "VISITANTE" in accion:
            return nombre_visitante
        return None

    df_cronologico["equipo_accion"] = df_cronologico.apply(identificar_equipo, axis=1)
    df_cronologico["tipo_equipo_accion"] = df_cronologico["equipo_accion"].apply(
        lambda eq: "local" if eq == nombre_local else ("visitante" if eq == nombre_visitante else None)
    )

    # === PASO 5: Finalizar ===
    df_cronologico = df_cronologico.drop(columns=['quinteto_en_cancha'])
    df_acciones_final = df_cronologico.iloc[::-1].reset_index(drop=True)

    print("\n✅ ¡Proceso completado! DataFrame final generado.")
    display(df_acciones_final.head(10))

except Exception as e:
    print(f"❌ Ocurrió un error en el script: {e}")

Iniciando el proceso completo para generar quintetos por equipo...
Paso 0: Identificando equipos local y visitante...
Equipo local: FERRO | Equipo visitante: PEÑAROL
Paso 1: Creando mapa de jugadores desde 'df_box_scores'...
Paso 2: Generando lista de 10 jugadores en cancha por acción...
Paso 3: Separando la lista de 10 jugadores en quintetos por equipo...
Paso 4B: Identificando equipo responsable de cada acción...

✅ ¡Proceso completado! DataFrame final generado.


,cuarto,tiempo,accion,jugador,puntos_local,puntos_visita,quinteto_local,quinteto_visita,equipo_local,equipo_visitante,equipo_accion,tipo_equipo_accion
0,,,FINAL DEL PERIODO5,,NaN,NaN,[],[],FERRO,PEÑAROL,None,None
1,5,00:00:01,REBOTE DEFENSIVO#9,"RODRIGUEZ, FELIPE SALVADOR",NaN,NaN,"[FIERRO, MARIANO, LADO, MOGGA CHARLES ARNADO, ...","[CHIARAVIGLIO, NICOLAS, FERNANDEZ, VICTOR LUIS...",FERRO,PEÑAROL,FERRO,local
2,5,00:00:02,TIRO DE 2 FALLADO,"WALLACE, DEVANTE RASHAD-KEITH",NaN,NaN,"[FIERRO, MARIANO, LADO, MOGGA CHARLES ARNADO, ...","[CHIARAVIGLIO, NICOLAS, FERNANDEZ, VICTOR LUIS...",FERRO,PEÑAROL,PEÑAROL,visitante
3,5,00:00:02,REBOTE OFENSIVO#24,"WALLACE, DEVANTE RASHAD-KEITH",NaN,NaN,"[FIERRO, MARIANO, LADO, MOGGA CHARLES ARNADO, ...","[CHIARAVIGLIO, NICOLAS, FERNANDEZ, VICTOR LUIS...",FERRO,PEÑAROL,PEÑAROL,visitante
4,5,00:00:02,TIRO LIBRE FALLADO,"THOMAS JR, MARCUS WILEY",NaN,NaN,"[FIERRO, MARIANO, LADO, MOGGA CHARLES ARNADO, ...","[CHIARAVIGLIO, NICOLAS, FERNANDEZ, VICTOR LUIS...",FERRO,PEÑAROL,PEÑAROL,visitante
5,5,00:00:02,TIRO LIBRE ANOTADO,"THOMAS JR, MARCUS WILEY",84.0,82.0,"[FIERRO, MARIANO, LADO, MOGGA CHARLES ARNADO, ...","[CHIARAVIGLIO, NICOLAS, FERNANDEZ, VICTOR LUIS...",FERRO,PEÑAROL,PEÑAROL,visitante
6,5,00:00:02,2 TIROS LIBRES PARA EL#50,"THOMAS JR, MARCUS WILEY",NaN,NaN,"[FIERRO, MARIANO, LADO, MOGGA CHARLES ARNADO, ...","[CHIARAVIGLIO, NICOLAS, FERNANDEZ, VICTOR LUIS...",FERRO,PEÑAROL,PEÑAROL,visitante
7,5,00:00:02,ENTRA A PISTA EL#93,"FIERRO, MARIANO",NaN,NaN,"[FIERRO, MARIANO, LADO, MOGGA CHARLES ARNADO, ...","[CHIARAVIGLIO, NICOLAS, FERNANDEZ, VICTOR LUIS...",FERRO,PEÑAROL,FERRO,local
8,5,00:00:02,ABANDONA LA PISTA EL#12,"GALLEGOS, RODRIGO AGUSTIN",NaN,NaN,"[LADO, MOGGA CHARLES ARNADO, LEZCANO, EMILIANO...","[CHIARAVIGLIO, NICOLAS, FERNANDEZ, VICTOR LUIS...",FERRO,PEÑAROL,FERRO,local
9,5,00:00:02,FALTA RECIBIDA,"THOMAS JR, MARCUS WILEY",NaN,NaN,"[GALLEGOS, RODRIGO AGUSTIN, LADO, MOGGA CHARLE...","[CHIARAVIGLIO, NICOLAS, FERNANDEZ, VICTOR LUIS...",FERRO,PEÑAROL,PEÑAROL,visitante


In [21]:
df_acciones_final[590:600
                  ]

,cuarto,tiempo,accion,jugador,puntos_local,puntos_visita,quinteto_local,quinteto_visita,equipo_local,equipo_visitante,equipo_accion,tipo_equipo_accion
590,1,00:09:33,TIRO DE 2 FALLADO,"THOMAS JR, MARCUS WILEY",NaN,NaN,"[FIERRO, MARIANO, GALLEGOS, RODRIGO AGUSTIN, L...","[BEDNAREK, IGNACIO NICOLAS, FERNANDEZ, VICTOR ...",FERRO,PEÑAROL,PEÑAROL,visitante
591,1,00:09:56,ASISTENCIA,"GALLEGOS, RODRIGO AGUSTIN",NaN,NaN,"[FIERRO, MARIANO, GALLEGOS, RODRIGO AGUSTIN, L...","[BEDNAREK, IGNACIO NICOLAS, FERNANDEZ, VICTOR ...",FERRO,PEÑAROL,FERRO,local
592,1,00:09:56,CANASTA DE 2 PUNTOS,"LEZCANO, EMILIANO",2.0,0.0,"[FIERRO, MARIANO, GALLEGOS, RODRIGO AGUSTIN, L...","[BEDNAREK, IGNACIO NICOLAS, FERNANDEZ, VICTOR ...",FERRO,PEÑAROL,FERRO,local
593,1,00:09:58,FLECHA ALTERNANCIA VISITANTE,,NaN,NaN,"[FIERRO, MARIANO, GALLEGOS, RODRIGO AGUSTIN, L...","[BEDNAREK, IGNACIO NICOLAS, FERNANDEZ, VICTOR ...",FERRO,PEÑAROL,PEÑAROL,visitante
594,,,INICIO DEL PERIODO1,,NaN,NaN,"[FIERRO, MARIANO, GALLEGOS, RODRIGO AGUSTIN, L...","[BEDNAREK, IGNACIO NICOLAS, FERNANDEZ, VICTOR ...",FERRO,PEÑAROL,None,None
595,1,00:10:00,ENTRA A PISTA EL#50,"THOMAS JR, MARCUS WILEY",NaN,NaN,"[FIERRO, MARIANO, GALLEGOS, RODRIGO AGUSTIN, L...","[BEDNAREK, IGNACIO NICOLAS, FERNANDEZ, VICTOR ...",FERRO,PEÑAROL,PEÑAROL,visitante
596,1,00:10:00,ENTRA A PISTA EL#18,"BEDNAREK, IGNACIO NICOLAS",NaN,NaN,"[FIERRO, MARIANO, GALLEGOS, RODRIGO AGUSTIN, L...","[BEDNAREK, IGNACIO NICOLAS, FERNANDEZ, VICTOR ...",FERRO,PEÑAROL,PEÑAROL,visitante
597,1,00:10:00,ENTRA A PISTA EL#2,"TOLOSA, FACUNDO GERMAN",NaN,NaN,"[FIERRO, MARIANO, GALLEGOS, RODRIGO AGUSTIN, L...","[FERNANDEZ, VICTOR LUIS, THORNTON, WILLIE ALFO...",FERRO,PEÑAROL,PEÑAROL,visitante
598,1,00:10:00,ENTRA A PISTA EL#1,"THORNTON, WILLIE ALFORD",NaN,NaN,"[FIERRO, MARIANO, GALLEGOS, RODRIGO AGUSTIN, L...","[FERNANDEZ, VICTOR LUIS, THORNTON, WILLIE ALFORD]",FERRO,PEÑAROL,PEÑAROL,visitante
599,1,00:10:00,ENTRA A PISTA EL#0,"FERNANDEZ, VICTOR LUIS",NaN,NaN,"[FIERRO, MARIANO, GALLEGOS, RODRIGO AGUSTIN, L...","[FERNANDEZ, VICTOR LUIS]",FERRO,PEÑAROL,PEÑAROL,visitante


In [22]:
df_acciones_final.to_excel('acciones_partido.xlsx', index=False)
print("DataFrame 'df_acciones_final' guardado como 'acciones_partido.xlsx'")

DataFrame 'df_acciones_final' guardado como 'acciones_partido.xlsx'
